In [1]:
import csv

In [3]:
input_data = '../data/OneSheet.csv'
data = []
with open(input_data,'r') as f:
    reader = csv.reader(f)
    heads = next(reader)
    for line in reader:
        data.append(line)

Class is final column

- 0 = original
- 1 = a
- 2 = b

In [18]:
import numpy as np
data = np.random.permutation(data)
N = len(data)
print(N)
fold_idxs = []
K = 10
fold_size = int(np.floor(N/K))
idx = list(range(N))
for k in range(K):
    this_fold = idx[k*fold_size:k*fold_size+fold_size]
    fold_idxs.append(this_fold)
fold_idxs[-1] += idx[K*fold_size:]

913
[91, 91, 91, 91, 91, 91, 91, 91, 91, 94]


Find unique AAs

In [24]:
AA_idx = {}
aa_pos = 0
for d in data:
    for aa in d[:4]:
        if not aa in AA_idx:
            AA_idx[aa] = aa_pos
            aa_pos += 1
print(AA_idx)

{'V': 0, 'G': 1, 'L': 2, 'I': 3, 'K': 4, 'T': 5, 'N': 6, 'E': 7, 'D': 8, 'Y': 9, 'H': 10, 'Q': 11, 'F': 12, 'C': 13, 'S': 14, 'A': 15, 'W': 16, 'M': 17, 'R': 18, 'P': 19}


In [48]:
classes = [0,1,2]
beta = 1 # smoothing parameter
pred_probs = []
pred_labels = []
for k in range(K):
    test_idx = fold_idxs[k]
    train_X = []
    test_X = []
    train_t = []
    test_t = []
    for n in range(N):
        if not n in test_idx:
            train_X.append(data[n][:4])
            train_t.append(int(data[n][4]))
        else:
            test_X.append(data[n][:4])
            test_t.append(int(data[n][4]))
    class_counts = {}
    for class_no in classes:
        class_counts[class_no] = [[beta for aa in AA_idx] for i in range(4)]
    for i,d in enumerate(train_X):
        class_no = train_t[i]
        for j,aa in enumerate(d):
            pos = AA_idx[aa]
            class_counts[class_no][j][pos] += 1
    class_probs = {}
    for class_no in classes:
        class_probs[class_no] = np.array(class_counts[class_no])
        class_probs[class_no] = class_probs[class_no]/class_probs[class_no].sum(axis=1)[:,None]

    prior_counts = {}
    for class_no in classes:
        prior_counts[class_no] = len(list(filter(lambda x: x == class_no,train_t))) / len(train_t)
    
    for i,d in enumerate(test_X):
        probs = []
        for class_no in classes:
            class_prob = 1
            for j in range(4):
                aa = d[j]
                aa_pos = AA_idx[aa]
                class_prob *= class_probs[class_no][j][aa_pos]
            probs.append(class_prob*prior_counts[class_no])
        probs = np.array(probs)
        probs /= probs.sum()
        pred_probs.append(probs)
        pred_labels.append(test_t[i])
    

In [60]:
correct = 0
raw_pred = []
for i,probs in enumerate(pred_probs):
    raw_pred.append(np.argmax(probs))
    if raw_pred[i] == pred_labels[i]:
        correct += 1
n_pred = len(raw_pred)
print(correct/n_pred)
confusion = np.zeros((3,3))
zz = list(zip(raw_pred,pred_labels))
for class_no in classes:
    for class_no2 in classes:
        confusion[class_no,class_no2] = len(list(filter(lambda x: x[0] == class_no and x[1] == class_no2,zz)))
print(confusion)

0.8795180722891566
[[419.  18.  20.]
 [ 18. 296.  35.]
 [  9.  10.  88.]]
